# Harmon: simple time series analysis (ignore other explanatory variables)

## Import required packges

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Read csv file into dataframes

In [2]:
harmon = pd.read_csv('Harmon exhibit 1.csv')
seasonality = pd.read_csv('Harmon seasonality index.csv')

## Check what data is in the dataframes

In [3]:
harmon.head()

,Month,Case Shipments,Consumer Packs,Dealer Allowance
0,1/1/1983,NaN,0,396776
1,2/1/1983,NaN,0,152296
2,3/1/1983,NaN,0,157640
3,4/1/1983,NaN,0,246064
4,5/1/1983,NaN,15012,335716


In [4]:
seasonality.head()

,Month,Seasonality Index
0,January,113
1,February,98
2,March,102
3,April,107
4,May,119


## Drop Consumer Packs and Dealer Allowance (don't need for simple time series)

In [5]:
harmon=harmon.drop(['Consumer Packs', 'Dealer Allowance'], axis=1)

## Convert Month column to pandas datetime format

In [6]:
harmon['Month']=harmon['Month'].apply(lambda x: pd.to_datetime(x))

### Convert Month to month of year (1 to 12)

In [7]:
harmon['Month of year'] = pd.DatetimeIndex(harmon['Month']).month

### Add month name

In [8]:
conditions2 = [harmon['Month of year'] == 1, 
              harmon['Month of year'] == 2,
              harmon['Month of year'] == 3,
              harmon['Month of year'] == 4,
              harmon['Month of year'] == 5,
              harmon['Month of year'] == 6,
              harmon['Month of year'] == 7,
              harmon['Month of year'] == 8,
              harmon['Month of year'] == 9,
              harmon['Month of year'] == 10,
              harmon['Month of year'] == 11,
              harmon['Month of year'] == 12,
             ]
choices2 = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
harmon['Month name'] = np.select(conditions2, choices2, default=0)

### Add a sequential month number

In [9]:
harmon['Seq month number']=harmon.index+1

### Rename columns in seasonality dataframe

In [10]:
seasonality=seasonality.rename(columns={"Month": "Month name"}, errors="raise")
seasonality=seasonality.rename(columns={"Seasonality Index": "Industry SI"}, errors="raise")

In [11]:
seasonality

,Month name,Industry SI
0,January,113
1,February,98
2,March,102
3,April,107
4,May,119
5,June,104
6,July,107
7,August,81
8,September,113
9,October,97


### Merge dataframes

In [12]:
harmon=pd.merge(harmon, seasonality, on=['Month name'])
harmon=harmon.sort_values(by=['Seq month number'])
harmon=harmon.reset_index()
harmon=harmon.drop(columns = ['index'])

### Check data

In [13]:
harmon.head()

,Month,Case Shipments,Month of year,Month name,Seq month number,Industry SI
0,1983-01-01,NaN,1,January,1,113
1,1983-02-01,NaN,2,February,2,98
2,1983-03-01,NaN,3,March,3,102
3,1983-04-01,NaN,4,April,4,107
4,1983-05-01,NaN,5,May,5,119


### Describe/summarize data in dataframes

In [14]:
harmon.describe()

,Case Shipments,Month of year,Seq month number,Industry SI
count,48.000000,60.000000,60.000000,60.000000
mean,382521.916667,6.500000,30.500000,100.083333
std,121169.648800,3.481184,17.464249,14.382536
min,166391.000000,1.000000,1.000000,65.000000
25%,298678.500000,3.750000,15.750000,96.500000
50%,371927.500000,6.500000,30.500000,103.000000
75%,437747.750000,9.250000,45.250000,108.500000
max,744583.000000,12.000000,60.000000,119.000000


### Drop missing values in data

In [15]:
harmon = harmon.dropna()

### Check data

In [16]:
harmon.head()

,Month,Case Shipments,Month of year,Month name,Seq month number,Industry SI
12,1984-01-01,425075.0,1,January,13,113
13,1984-02-01,315305.0,2,February,14,98
14,1984-03-01,367286.0,3,March,15,102
15,1984-04-01,429432.0,4,April,16,107
16,1984-05-01,347874.0,5,May,17,119


## Simple time series analysis

### Basic scatter plot

In [18]:
fig = px.scatter(harmon, x="Seq month number", y="Case Shipments", trendline="ols")
fig.show()
## More basic scatter plot
#plt.style.use('seaborn')
#harmon.plot(x='Seq month number', y='Case Shipments', kind='scatter')
#plt.show()

### Basic regression to find level and trend 

#### Note that statsmodels does not add an intercept; need to "add.constant"

In [19]:
X = harmon['Seq month number']
X = sm.add_constant(X)
y = harmon['Case Shipments']
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         Case Shipments   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.046
Date:                Wed, 19 Jan 2022   Prob (F-statistic):              0.312
Time:                        11:11:46   Log-Likelihood:                -628.90
No. Observations:                  48   AIC:                             1262.
Df Residuals:                      46   BIC:                             1266.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             3.354e+05   4.93e+04  

### Deseasonalize data

In [20]:
harmon['Deseasonalized Case Shipments']=harmon['Case Shipments'].rolling(window=12, center=True).mean().rolling(2).mean().shift(-1)

### Check data

In [21]:
harmon.head()

,Month,Case Shipments,Month of year,Month name,Seq month number,Industry SI,Deseasonalized Case Shipments
12,1984-01-01,425075.0,1,January,13,113,NaN
13,1984-02-01,315305.0,2,February,14,98,NaN
14,1984-03-01,367286.0,3,March,15,102,NaN
15,1984-04-01,429432.0,4,April,16,107,NaN
16,1984-05-01,347874.0,5,May,17,119,NaN


### Drop missing values

In [22]:
harmon_deseasonalized=harmon.dropna()

### Check data

In [23]:
harmon_deseasonalized.head()

,Month,Case Shipments,Month of year,Month name,Seq month number,Industry SI,Deseasonalized Case Shipments
18,1984-07-01,299403.0,7,July,19,107,351930.791667
19,1984-08-01,296505.0,8,August,20,81,358284.500000
20,1984-09-01,426701.0,9,September,21,113,357417.666667
21,1984-10-01,329722.0,10,October,22,97,356508.125000
22,1984-11-01,281783.0,11,November,23,95,358327.583333


### Basic deseasonalized scatter plot

In [24]:
fig = px.scatter(harmon_deseasonalized, x="Seq month number", y="Deseasonalized Case Shipments", trendline="ols")
fig.show()

### Basic deseasonalized regression to find level and trend

In [25]:
X = harmon_deseasonalized['Seq month number']
X = sm.add_constant(X)
y = harmon_deseasonalized['Deseasonalized Case Shipments']
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                                  OLS Regression Results                                 
Dep. Variable:     Deseasonalized Case Shipments   R-squared:                       0.891
Model:                                       OLS   Adj. R-squared:                  0.887
Method:                            Least Squares   F-statistic:                     276.7
Date:                           Wed, 19 Jan 2022   Prob (F-statistic):           6.65e-18
Time:                                   11:13:32   Log-Likelihood:                -363.45
No. Observations:                             36   AIC:                             730.9
Df Residuals:                                 34   BIC:                             734.1
Df Model:                                      1                                         
Covariance Type:                       nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
---------------

### Predict deseasonalized demand using above model

In [26]:
harmon['M1 Predicted Deseasonalized Case Shipments'] = results.predict(X)

### Check data

In [27]:
harmon

,Month,Case Shipments,Month of year,Month name,Seq month number,Industry SI,Deseasonalized Case Shipments,M1 Predicted Deseasonalized Case Shipments
12,1984-01-01,425075.0,1,January,13,113,NaN,NaN
13,1984-02-01,315305.0,2,February,14,98,NaN,NaN
14,1984-03-01,367286.0,3,March,15,102,NaN,NaN
15,1984-04-01,429432.0,4,April,16,107,NaN,NaN
16,1984-05-01,347874.0,5,May,17,119,NaN,NaN
17,1984-06-01,435529.0,6,June,18,104,NaN,NaN
18,1984-07-01,299403.0,7,July,19,107,351930.791667,351183.773211
19,1984-08-01,296505.0,8,August,20,81,358284.500000,352794.414808
20,1984-09-01,426701.0,9,September,21,113,357417.666667,354405.056405
21,1984-10-01,329722.0,10,October,22,97,356508.125000,356015.698002


### Put seasonal effects back in (using industry provided data)

In [28]:
harmon['M1 Predicted Case Shipments'] = harmon['M1 Predicted Deseasonalized Case Shipments']*(harmon['Industry SI']/100)

### Check data

In [29]:
harmon

,Month,Case Shipments,Month of year,Month name,Seq month number,Industry SI,Deseasonalized Case Shipments,M1 Predicted Deseasonalized Case Shipments,M1 Predicted Case Shipments
12,1984-01-01,425075.0,1,January,13,113,NaN,NaN,NaN
13,1984-02-01,315305.0,2,February,14,98,NaN,NaN,NaN
14,1984-03-01,367286.0,3,March,15,102,NaN,NaN,NaN
15,1984-04-01,429432.0,4,April,16,107,NaN,NaN,NaN
16,1984-05-01,347874.0,5,May,17,119,NaN,NaN,NaN
17,1984-06-01,435529.0,6,June,18,104,NaN,NaN,NaN
18,1984-07-01,299403.0,7,July,19,107,351930.791667,351183.773211,375766.637335
19,1984-08-01,296505.0,8,August,20,81,358284.500000,352794.414808,285763.475994
20,1984-09-01,426701.0,9,September,21,113,357417.666667,354405.056405,400477.713737
21,1984-10-01,329722.0,10,October,22,97,356508.125000,356015.698002,345335.227062


### Calculate Abs Error

In [30]:
harmon['M1 Abs Error']= abs(harmon['M1 Predicted Case Shipments'] - harmon['Case Shipments'])

### Check data

In [31]:
harmon

,Month,Case Shipments,Month of year,Month name,Seq month number,Industry SI,Deseasonalized Case Shipments,M1 Predicted Deseasonalized Case Shipments,M1 Predicted Case Shipments,M1 Abs Error
12,1984-01-01,425075.0,1,January,13,113,NaN,NaN,NaN,NaN
13,1984-02-01,315305.0,2,February,14,98,NaN,NaN,NaN,NaN
14,1984-03-01,367286.0,3,March,15,102,NaN,NaN,NaN,NaN
15,1984-04-01,429432.0,4,April,16,107,NaN,NaN,NaN,NaN
16,1984-05-01,347874.0,5,May,17,119,NaN,NaN,NaN,NaN
17,1984-06-01,435529.0,6,June,18,104,NaN,NaN,NaN,NaN
18,1984-07-01,299403.0,7,July,19,107,351930.791667,351183.773211,375766.637335,76363.637335
19,1984-08-01,296505.0,8,August,20,81,358284.500000,352794.414808,285763.475994,10741.524006
20,1984-09-01,426701.0,9,September,21,113,357417.666667,354405.056405,400477.713737,26223.286263
21,1984-10-01,329722.0,10,October,22,97,356508.125000,356015.698002,345335.227062,15613.227062


### Plot case shipments and forecasted case shipments

In [32]:
x=harmon['Seq month number']
y1=harmon['Case Shipments']
y2=harmon['M1 Predicted Case Shipments']
#fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=x, y=y1,
                    mode='lines+markers', name='Case Shipments'), secondary_y=False)
fig.add_trace(go.Scatter(x=x, y=y2,
                    mode='lines+markers', name='M1 Predicted Case Shipments'), secondary_y=False)
fig.show()

## Time series with data-driven seasonal index

### Create data driven seasonal factors

In [33]:
harmon['Data-driven seasonal factor']=(harmon['Case Shipments']/harmon['Deseasonalized Case Shipments'])*100

### Check data

In [34]:
harmon

,Month,Case Shipments,Month of year,Month name,Seq month number,Industry SI,Deseasonalized Case Shipments,M1 Predicted Deseasonalized Case Shipments,M1 Predicted Case Shipments,M1 Abs Error,Data-driven seasonal factor
12,1984-01-01,425075.0,1,January,13,113,NaN,NaN,NaN,NaN,NaN
13,1984-02-01,315305.0,2,February,14,98,NaN,NaN,NaN,NaN,NaN
14,1984-03-01,367286.0,3,March,15,102,NaN,NaN,NaN,NaN,NaN
15,1984-04-01,429432.0,4,April,16,107,NaN,NaN,NaN,NaN,NaN
16,1984-05-01,347874.0,5,May,17,119,NaN,NaN,NaN,NaN,NaN
17,1984-06-01,435529.0,6,June,18,104,NaN,NaN,NaN,NaN,NaN
18,1984-07-01,299403.0,7,July,19,107,351930.791667,351183.773211,375766.637335,76363.637335,85.074397
19,1984-08-01,296505.0,8,August,20,81,358284.500000,352794.414808,285763.475994,10741.524006,82.756859
20,1984-09-01,426701.0,9,September,21,113,357417.666667,354405.056405,400477.713737,26223.286263,119.384418
21,1984-10-01,329722.0,10,October,22,97,356508.125000,356015.698002,345335.227062,15613.227062,92.486532


### Use Pivot to calculate data-driven seasonal index (average of seasonal factors for each month)

In [35]:
datadrivenSI = pd.pivot_table(harmon, values='Data-driven seasonal factor', index=['Month of year'], aggfunc=np.mean)
datadrivenSI=datadrivenSI.rename(columns={"Data-driven seasonal factor": "Data-driven SI"}, errors="raise")
datadrivenSI

,Data-driven SI
Month of year,
1,154.602095
2,68.887757
3,119.966627
4,95.971315
5,121.894583
6,92.577138
7,106.286339
8,72.757960
9,130.764380


### Merge this back into the dataframe

In [36]:
harmon=pd.merge(harmon, datadrivenSI, on=['Month of year'])
harmon=harmon.sort_values(by=['Seq month number'])
harmon=harmon.reset_index()
harmon=harmon.drop(columns = ['index'])

### Check data

In [37]:
harmon

,Month,Case Shipments,Month of year,Month name,Seq month number,Industry SI,Deseasonalized Case Shipments,M1 Predicted Deseasonalized Case Shipments,M1 Predicted Case Shipments,M1 Abs Error,Data-driven seasonal factor,Data-driven SI
0,1984-01-01,425075.0,1,January,13,113,NaN,NaN,NaN,NaN,NaN,154.602095
1,1984-02-01,315305.0,2,February,14,98,NaN,NaN,NaN,NaN,NaN,68.887757
2,1984-03-01,367286.0,3,March,15,102,NaN,NaN,NaN,NaN,NaN,119.966627
3,1984-04-01,429432.0,4,April,16,107,NaN,NaN,NaN,NaN,NaN,95.971315
4,1984-05-01,347874.0,5,May,17,119,NaN,NaN,NaN,NaN,NaN,121.894583
5,1984-06-01,435529.0,6,June,18,104,NaN,NaN,NaN,NaN,NaN,92.577138
6,1984-07-01,299403.0,7,July,19,107,351930.791667,351183.773211,375766.637335,76363.637335,85.074397,106.286339
7,1984-08-01,296505.0,8,August,20,81,358284.500000,352794.414808,285763.475994,10741.524006,82.756859,72.757960
8,1984-09-01,426701.0,9,September,21,113,357417.666667,354405.056405,400477.713737,26223.286263,119.384418,130.764380
9,1984-10-01,329722.0,10,October,22,97,356508.125000,356015.698002,345335.227062,15613.227062,92.486532,86.392592


### Calculate predicted case shipments based the data-driven SI

In [38]:
harmon['M2 Predicted Case Shipments'] = harmon['M1 Predicted Deseasonalized Case Shipments']*(harmon['Data-driven SI']/100)

### Check data

In [39]:
harmon

,Month,Case Shipments,Month of year,Month name,Seq month number,Industry SI,Deseasonalized Case Shipments,M1 Predicted Deseasonalized Case Shipments,M1 Predicted Case Shipments,M1 Abs Error,Data-driven seasonal factor,Data-driven SI,M2 Predicted Case Shipments
0,1984-01-01,425075.0,1,January,13,113,NaN,NaN,NaN,NaN,NaN,154.602095,NaN
1,1984-02-01,315305.0,2,February,14,98,NaN,NaN,NaN,NaN,NaN,68.887757,NaN
2,1984-03-01,367286.0,3,March,15,102,NaN,NaN,NaN,NaN,NaN,119.966627,NaN
3,1984-04-01,429432.0,4,April,16,107,NaN,NaN,NaN,NaN,NaN,95.971315,NaN
4,1984-05-01,347874.0,5,May,17,119,NaN,NaN,NaN,NaN,NaN,121.894583,NaN
5,1984-06-01,435529.0,6,June,18,104,NaN,NaN,NaN,NaN,NaN,92.577138,NaN
6,1984-07-01,299403.0,7,July,19,107,351930.791667,351183.773211,375766.637335,76363.637335,85.074397,106.286339,373260.374015
7,1984-08-01,296505.0,8,August,20,81,358284.500000,352794.414808,285763.475994,10741.524006,82.756859,72.757960,256686.019972
8,1984-09-01,426701.0,9,September,21,113,357417.666667,354405.056405,400477.713737,26223.286263,119.384418,130.764380,463435.574502
9,1984-10-01,329722.0,10,October,22,97,356508.125000,356015.698002,345335.227062,15613.227062,92.486532,86.392592,307571.190593


### Calculate absolute error

In [40]:
harmon['M2 Abs Error']= abs(harmon['M2 Predicted Case Shipments'] - harmon['Case Shipments'])

### Plot case shipments and prediction from data-driven SI model

In [41]:
x=harmon['Seq month number']
y1=harmon['Case Shipments']
y2=harmon['M2 Predicted Case Shipments']
#fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=x, y=y1,
                    mode='lines+markers', name='Case Shipments'), secondary_y=False)
fig.add_trace(go.Scatter(x=x, y=y2,
                    mode='lines+markers', name='M2 Predicted Case Shipments'), secondary_y=False)
fig.show()

### Plot data on case shipments and predictions from both models

In [42]:
x=harmon['Seq month number']
y1=harmon['Case Shipments']
y2=harmon['M1 Predicted Case Shipments']
y3=harmon['M2 Predicted Case Shipments']
#fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=x, y=y1,
                    mode='lines+markers', name='Case Shipments'), secondary_y=False)
fig.add_trace(go.Scatter(x=x, y=y2,
                    mode='lines+markers', name='M1 Predicted Case Shipments'), secondary_y=False)
fig.add_trace(go.Scatter(x=x, y=y3,
                    mode='lines+markers', name='M2 Predicted Case Shipments'), secondary_y=False)
fig.show()

### Compare absolute errors

#### Model 1 Mean Abs Error

In [43]:
harmon['M1 Abs Error'].mean()

63943.98777710001

#### Model 2 Mean Abs Error

In [44]:
harmon['M2 Abs Error'].mean()

40831.790517089394